# Artificial Neural Network

### Importing the libraries

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [2]:
tf.__version__

'2.11.0'

## Part 1 - Data Preprocessing

### Importing the dataset

In [3]:
df_train=pd.read_csv('train.csv')
df_test=pd.read_csv('test.csv')

In [6]:
df_Xtrain= df_train.loc[:,df_train.columns.difference(['Machine failure','Product ID','id'])]

In [7]:
df_Xtrain.head()

,Air temperature [K],HDF,OSF,PWF,Process temperature [K],RNF,Rotational speed [rpm],TWF,Tool wear [min],Torque [Nm],Type
0,300.6,0,0,0,309.6,0,1596,0,140,36.1,L
1,302.6,0,0,0,312.1,0,1759,0,200,29.1,M
2,299.3,0,0,0,308.5,0,1805,0,25,26.5,L
3,301.0,0,0,0,310.9,0,1524,0,197,44.3,L
4,298.0,0,0,0,309.0,0,1641,0,34,35.4,M


In [8]:
df_ytrain= df_train.loc[:,df_train.columns=='Machine failure']

In [9]:
df_ytrain.head()

,Machine failure
0,0
1,0
2,0
3,0
4,0


### Encoding categorical data

Label Encoding the "Gender" column

In [0]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [-1])], remainder='passthrough')
df_Xtrain = np.array(ct.fit_transform(df_Xtrain))

In [ ]:
df_Xtrain[0]

### Splitting the dataset into the Training set and Test set

In [12]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df_Xtrain, df_ytrain, test_size = 0.2, random_state = 42)

### Feature Scaling

In [0]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

## Part 2 - Building the ANN

### Initializing the ANN

In [13]:
ann = tf.keras.models.Sequential()

### Adding the input layer and the first hidden layer

In [14]:
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))

### Adding the second hidden layer

In [15]:
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))

### Adding the output layer

In [16]:
ann.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

## Part 3 - Training the ANN

### Compiling the ANN

In [17]:
ann.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

### Training the ANN on the Training set

In [18]:
ann.fit(X_train, y_train, batch_size = 32, epochs = 100)


#Building part was our forst half of to creating ANN now the second part is to train our ANN we are goona do this in two steps
#!st is to compile ANN with an optimizer loss function and a metric, wich will be ofcorse the accuracy 
#because we re doung some classifiaction
#Then the 2n step ll be of course to train ANN on the training set over a certain number of epocs
#we use use here keras again inteadrated with tf now because w dont add any thing in our ANN , we compile it so we use .compile
#ann.compile(-->.3 parametere) 1- optimizer to choose an optimizer 2= loss to chose a loss function 3 is metric's' parameter
#with 's' because there are several metrics but we chosse only one
#Well, in the intuition lectures, Kirel mentioned that the best one are the optimizers that can perform
#stochastic gradient descent.

Epoch 1/100
3411/3411 [==============================] - 12s 3ms/step - loss: 9.6057 - accuracy: 0.9388
Epoch 2/100
3411/3411 [==============================] - 9s 3ms/step - loss: 0.0897 - accuracy: 0.9840
Epoch 3/100
3411/3411 [==============================] - 8s 2ms/step - loss: 0.0836 - accuracy: 0.9843
Epoch 4/100
3411/3411 [==============================] - 10s 3ms/step - loss: 0.0748 - accuracy: 0.9843
Epoch 5/100
3411/3411 [==============================] - 12s 4ms/step - loss: 0.0615 - accuracy: 0.9851
Epoch 6/100
3411/3411 [==============================] - 9s 3ms/step - loss: 0.0564 - accuracy: 0.9862
Epoch 7/100
3411/3411 [==============================] - 8s 2ms/step - loss: 0.0513 - accuracy: 0.9873
Epoch 8/100
3411/3411 [==============================] - 8s 2ms/step - loss: 0.0437 - accuracy: 0.9894
Epoch 9/100
3411/3411 [==============================] - 8s 2ms/step - loss: 0.0363 - accuracy: 0.9921
Epoch 10/100
3411/3411 [==============================] - 8s 2ms/step 

## Part 4 - Making the predictions and evaluating the model

### Predicting the result of a single observation

**Homework**

Use our ANN model to predict if the customer with the following informations will leave the bank: 

Geography: France

Credit Score: 600

Gender: Male

Age: 40 years old

Tenure: 3 years

Balance: \$ 60000

Number of Products: 2

Does this customer have a credit card? Yes

Is this customer an Active Member: Yes

Estimated Salary: \$ 50000

So, should we say goodbye to that customer?

**Solution**

In [18]:
print(ann.predict(sc.transform([[1, 0, 0, 600, 1, 40, 3, 60000, 2, 1, 1, 50000]])) > 0.5)

[[False]]


Therefore, our ANN model predicts that this customer stays in the bank!

**Important note 1:** Notice that the values of the features were all input in a double pair of square brackets. That's because the "predict" method always expects a 2D array as the format of its inputs. And putting our values into a double pair of square brackets makes the input exactly a 2D array.

**Important note 2:** Notice also that the "France" country was not input as a string in the last column but as "1, 0, 0" in the first three columns. That's because of course the predict method expects the one-hot-encoded values of the state, and as we see in the first row of the matrix of features X, "France" was encoded as "1, 0, 0". And be careful to include these values in the first three columns, because the dummy variables are always created in the first columns.

### Predicting the Test set results

In [21]:
y_pred = ann.predict(X_test)
y_pred = (y_pred > 0.5)
print(np.concatenate((np.array(y_pred).reshape(len(y_pred),1), np.array(y_test).reshape(len(y_test),1)),1))

853/853 [==============================] - 1s 2ms/step
[[1 1]
 [0 0]
 [0 0]
 ...
 [0 0]
 [0 0]
 [0 0]]


In [25]:
y_pred = ann.predict(X_test)
y_pred = (y_pred > 0.8)
print(np.concatenate((np.array(y_pred).reshape(len(y_pred),1), np.array(y_test).reshape(len(y_test),1)),1))

853/853 [==============================] - 3s 3ms/step
[[0 1]
 [0 0]
 [0 0]
 ...
 [0 0]
 [0 0]
 [0 0]]


In [26]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[26847     1]
 [  209   229]]


0.9923037455105183

### Making the Confusion Matrix

In [22]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[26847     1]
 [  138   300]]


0.9949058125045811